# Attack Password with Correlation Power Analysis IV.1 (CPA)

In [ ]:
%run '../util/Metadata.ipynb'
print_metadata()

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Improving-the-code" data-toc-modified-id="Improving-the-code-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Improving the code</a></span></li><li><span><a href="#Basic-Setup" data-toc-modified-id="Basic-Setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Basic Setup</a></span></li><li><span><a href="#Helper-Functions-for-Password-Attack" data-toc-modified-id="Helper-Functions-for-Password-Attack-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Helper Functions for Password Attack</a></span></li><li><span><a href="#New-Code---Old-results" data-toc-modified-id="New-Code---Old-results-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>New Code - Old results</a></span></li><li><span><a href="#Disconnect" data-toc-modified-id="Disconnect-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Disconnect</a></span></li></ul></div>

In this example we want to discuss possibilities to fight against a CPA attack.

## Improving the code

Let's first recap the password checking loop from `advanced-passwdcheck-xor`:
```c
passbad = 0;
for(uint8_t i = 0; i < sizeof(correct_passwd); i++){
    passbad |= correct_passwd[i] ^ passwd[i];
}
```

We revealed in the last example that aboves XOR generates a collision between known and determinable input data and secret data which shall be revealed by an attack.

So, how to lower this correlation?

## Basic Setup

Define Variables

In [ ]:
%run "../util/Init.ipynb"

Build target and upload

In [ ]:
TARGET = 'simpleserial-passwordcheck'
%store TARGET
%run "$HELPERSCRIPTS/Prepare.ipynb"

Import helper functions

In [ ]:
%run "$HELPERSCRIPTS/Setup_Generic.ipynb"

In [ ]:
scope.adc.samples = 500

## Helper Functions for Password Attack

In [ ]:
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook
from bokeh.models import CrosshairTool, Label

output_notebook()

In [ ]:
import warnings
import random
import tqdm
import numpy as np

def capture(command, data):
    scope.arm()
    target.simpleserial_write(command, data)

    ret = scope.capture()

    i = 0
    while not target.is_done():
        i += 1
        time.sleep(0.05)
        if i > 100:
            warnings.warn("Target did not finish operation")
            return None

    if ret:
        warnings.warn("Timeout happened during capture")
        return None

    return scope.get_last_trace()

def pad_password(password, length):
    return (password + b'\x00' * length)[:length]

def target_set_random(random_length=32):
    target.simpleserial_write('r', bytes(random.choices(range(0, 256), k=random_length)))

def target_set_password(password, max_password_length=8):
    target.simpleserial_write('p', pad_password(password, max_password_length))
    return target.simpleserial_read('r', max_password_length)

def target_check_password(command, password, max_password_length=8):
    target.simpleserial_write(command, pad_password(password, max_password_length))
    return bytes(target.simpleserial_read('r', 1))[0] == 0

def capture_random(command, size=500, password_length=8):
    """Collect size number of password attempts with fully random random data."""
    traces = []
    textins = []
    for _ in tqdm.tqdm_notebook(range(size)):
        target_set_random()
        pass_guess = bytes(random.choices(range(0, 256), k=password_length))
        traces.append(capture(command, pass_guess))
        textins.append(pass_guess)
    return np.array(traces), textins

In [ ]:
import numpy as np

HW = [bin(n).count("1") for n in range(0, 256)]

def hw(n):
    return HW[n]

hw_vec = np.vectorize(hw)

def pearson(x: np.array, y: np.array):
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    return sum((x - x_mean) * (y - y_mean)) / np.sqrt(sum((x - x_mean) ** 2) * sum((y - y_mean) ** 2))

def pearson_pointwise(traces, intermediates):
    intermediates_diff = intermediates - np.mean(intermediates)
    intermediates_sqrt = np.sqrt(np.sum(intermediates_diff ** 2))
    traces_diff = traces - np.mean(traces, axis=0)
    
    return np.sum(traces_diff * intermediates_diff[:, None], axis=0) / (
        np.sqrt(np.sum(traces_diff ** 2, axis=0)) * intermediates_sqrt
    )

In [ ]:
import bokeh.palettes
import bokeh.transform
from bokeh.models import ColumnDataSource

def correlation_plot(correlations, color_palette=bokeh.palettes.Oranges6, **kw):
    kw['height'] = kw.get('height', 300)
    kw['y_range'] = kw.get('y_range', (-1, 1))
    p = figure(sizing_mode='stretch_width', **kw)
    p.vbar(
        x='points',
        top='corr',
        width=1,
        source=ColumnDataSource(data=dict(
            points=range(len(correlations)),
            corr=correlations,
            abscorr=abs(correlations),
        )),
        color=bokeh.transform.linear_cmap(
            field_name='abscorr', 
            palette=color_palette,
            low=1,
            high=0,
        ),
    )
    return p

## New Code - Old results

In [ ]:
reset_target(scope)
password = b'ifx2019'
target_set_password(password)
print('password "{}" is correct?'.format(password.decode()), target_check_password('1', password))
traces, textins = capture_random(command='1', size=200, password_length=8)
correlations = pearson_pointwise(traces, [hw(textin[0] ^ ord('i')) for textin in textins])
show(correlation_plot(correlations, x_range=(0, 100)))

## Disconnect

In [ ]:
scope.dis()
target.dis()